# 🎯 Task 6: Object Detection Concepts and Metrics

## 🎯 Objective
Implement and understand all key object detection metrics used in YOLO evaluation.

### ML Rules Applied:
- **Rule #2**: First, design and implement metrics
- **Rule #13**: Choose a simple, observable metric
- **Rule #24**: Measure the delta between models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path

np.random.seed(42)
PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
print("✅ Libraries imported (NumPy only!)")

---

# Part 1: Intersection over Union (IoU)

## 📐 Mathematical Definition

IoU measures the overlap between two bounding boxes:

$$
IoU = \frac{|A \cap B|}{|A \cup B|} = \frac{\text{Intersection Area}}{\text{Union Area}}
$$

$$
IoU = \frac{\text{Intersection}}{\text{Area}_A + \text{Area}_B - \text{Intersection}}
$$

**IoU Properties:**
- Range: [0, 1]
- IoU = 1: Perfect overlap
- IoU = 0: No overlap
- IoU ≥ 0.5: Typically considered a "match"

In [ ]:
# ============================================================
# IoU CALCULATION - NumPy Implementation
# ============================================================

def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union between two boxes.
    
    Formula: IoU = Intersection / Union
    
    Args:
        box1, box2: [x1, y1, x2, y2] format (corners)
    
    Returns:
        IoU value in [0, 1]
    """
    # Unpack boxes
    x1_a, y1_a, x2_a, y2_a = box1
    x1_b, y1_b, x2_b, y2_b = box2
    
    # Calculate intersection coordinates
    x1_inter = max(x1_a, x1_b)
    y1_inter = max(y1_a, y1_b)
    x2_inter = min(x2_a, x2_b)
    y2_inter = min(y2_a, y2_b)
    
    # Calculate intersection area
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    intersection = inter_width * inter_height
    
    # Calculate union area
    area_a = (x2_a - x1_a) * (y2_a - y1_a)
    area_b = (x2_b - x1_b) * (y2_b - y1_b)
    union = area_a + area_b - intersection
    
    # Calculate IoU
    iou = intersection / (union + 1e-6)  # Avoid division by zero
    
    return iou

# Test
box_gt = [100, 100, 200, 200]   # Ground truth
box_pred = [120, 110, 210, 210] # Prediction (partial overlap)
iou = calculate_iou(box_gt, box_pred)
print(f"✅ IoU between boxes: {iou:.4f}")

In [ ]:
# Visualize IoU
def visualize_iou(box1, box2):
    """Visualize IoU between two boxes."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Draw boxes
    rect1 = patches.Rectangle((box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
                              linewidth=3, edgecolor='green', facecolor='green', alpha=0.3, label='Ground Truth')
    rect2 = patches.Rectangle((box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
                              linewidth=3, edgecolor='blue', facecolor='blue', alpha=0.3, label='Prediction')
    
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    
    # Calculate and display IoU
    iou = calculate_iou(box1, box2)
    
    ax.set_xlim(0, 300)
    ax.set_ylim(300, 0)  # Invert y
    ax.set_title(f'IoU Visualization\nIoU = {iou:.4f}', fontsize=14, fontweight='bold')
    ax.legend()
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'iou_visualization.png', dpi=150)
    plt.show()

visualize_iou(box_gt, box_pred)

---

# Part 2: Non-Maximum Suppression (NMS)

## 📐 Algorithm

NMS removes redundant overlapping detections:

```
1. Sort all boxes by confidence score (descending)
2. Select box with highest confidence
3. Remove all boxes with IoU > threshold with selected box
4. Repeat until no boxes remain
```

**Why needed:** A single object may trigger multiple detections.

In [ ]:
# ============================================================
# NON-MAXIMUM SUPPRESSION - NumPy Implementation
# ============================================================

def nms(boxes, scores, iou_threshold=0.5):
    """
    Non-Maximum Suppression (NumPy only).
    
    Args:
        boxes: Array of [x1, y1, x2, y2] boxes, shape (N, 4)
        scores: Confidence scores for each box, shape (N,)
        iou_threshold: IoU threshold for suppression
    
    Returns:
        Indices of boxes to keep
    """
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes)
    scores = np.array(scores)
    
    # Sort by confidence (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    keep = []
    
    while len(sorted_indices) > 0:
        # Keep the box with highest confidence
        best_idx = sorted_indices[0]
        keep.append(best_idx)
        
        # Remove it from consideration
        sorted_indices = sorted_indices[1:]
        
        if len(sorted_indices) == 0:
            break
        
        # Calculate IoU with remaining boxes
        remaining_boxes = boxes[sorted_indices]
        ious = np.array([calculate_iou(boxes[best_idx], box) for box in remaining_boxes])
        
        # Keep only boxes with IoU below threshold
        mask = ious < iou_threshold
        sorted_indices = sorted_indices[mask]
    
    return keep

# Test NMS
test_boxes = [
    [100, 100, 200, 200],
    [110, 105, 205, 205],  # Overlaps heavily
    [115, 110, 210, 210],  # Overlaps heavily
    [300, 300, 400, 400],  # Different object
]
test_scores = [0.9, 0.75, 0.8, 0.85]

keep_indices = nms(test_boxes, test_scores, iou_threshold=0.5)
print(f"✅ NMS kept {len(keep_indices)} out of {len(test_boxes)} boxes")
print(f"   Kept indices: {keep_indices}")

In [ ]:
# Visualize NMS
def visualize_nms(boxes, scores, keep_indices):
    """Visualize NMS before and after."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(boxes)))
    
    # Before NMS
    ax1 = axes[0]
    for i, (box, score) in enumerate(zip(boxes, scores)):
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                                 linewidth=2, edgecolor=colors[i], facecolor='none')
        ax1.add_patch(rect)
        ax1.text(box[0], box[1]-5, f'{score:.2f}', fontsize=10, color=colors[i])
    ax1.set_xlim(50, 450)
    ax1.set_ylim(450, 50)
    ax1.set_title(f'Before NMS ({len(boxes)} boxes)', fontsize=12, fontweight='bold')
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # After NMS
    ax2 = axes[1]
    for i in keep_indices:
        box = boxes[i]
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                                 linewidth=3, edgecolor='green', facecolor='green', alpha=0.3)
        ax2.add_patch(rect)
        ax2.text(box[0], box[1]-5, f'{scores[i]:.2f}', fontsize=10, color='green', fontweight='bold')
    ax2.set_xlim(50, 450)
    ax2.set_ylim(450, 50)
    ax2.set_title(f'After NMS ({len(keep_indices)} boxes)', fontsize=12, fontweight='bold')
    ax2.set_aspect('equal')
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('🎯 Non-Maximum Suppression (NMS)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'nms_demo.png', dpi=150)
    plt.show()

visualize_nms(test_boxes, test_scores, keep_indices)

---

# Part 3: Precision, Recall, F1-Score

## 📐 Mathematical Definitions

$$
\text{Precision} = \frac{TP}{TP + FP} = \frac{\text{Correct Detections}}{\text{All Detections}}
$$

$$
\text{Recall} = \frac{TP}{TP + FN} = \frac{\text{Correct Detections}}{\text{All Ground Truths}}
$$

$$
\text{F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Where:
- **TP** (True Positive): Correct detection (IoU ≥ threshold)
- **FP** (False Positive): Wrong detection (no matching GT)
- **FN** (False Negative): Missed object (GT not detected)

In [ ]:
# ============================================================
# PRECISION, RECALL, F1 - NumPy Implementation
# ============================================================

def calculate_precision_recall(pred_boxes, gt_boxes, iou_threshold=0.5):
    """
    Calculate precision and recall for object detection.
    
    Args:
        pred_boxes: List of predicted boxes [x1, y1, x2, y2]
        gt_boxes: List of ground truth boxes
        iou_threshold: IoU threshold for matching
    
    Returns:
        precision, recall, f1_score
    """
    if len(pred_boxes) == 0:
        return 0, 0, 0
    if len(gt_boxes) == 0:
        return 0, 0, 0
    
    # Track which GT boxes have been matched
    gt_matched = [False] * len(gt_boxes)
    
    tp = 0  # True positives
    fp = 0  # False positives
    
    for pred_box in pred_boxes:
        best_iou = 0
        best_gt_idx = -1
        
        # Find best matching GT box
        for gt_idx, gt_box in enumerate(gt_boxes):
            if gt_matched[gt_idx]:
                continue
            
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        
        # Check if match is good enough
        if best_iou >= iou_threshold:
            tp += 1
            gt_matched[best_gt_idx] = True
        else:
            fp += 1
    
    fn = sum(1 for matched in gt_matched if not matched)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Test
gt_boxes = [[100, 100, 200, 200], [300, 300, 400, 400]]
pred_boxes = [[105, 105, 205, 205], [305, 305, 395, 395], [500, 500, 600, 600]]  # 2 good, 1 FP

p, r, f1 = calculate_precision_recall(pred_boxes, gt_boxes)
print(f"✅ Precision: {p:.4f}")
print(f"   Recall: {r:.4f}")
print(f"   F1-Score: {f1:.4f}")

---

# Part 4: Mean Average Precision (mAP)

## 📐 Mathematical Definition

**Average Precision (AP)** for one class:
$$
AP = \int_0^1 p(r) \, dr \approx \sum_{n} (r_{n+1} - r_n) \cdot p_{interp}(r_{n+1})
$$

**Mean Average Precision**:
$$
mAP = \frac{1}{|C|} \sum_{c \in C} AP_c
$$

**Common Metrics:**
- **mAP@50**: Using IoU threshold = 0.5
- **mAP@50-95**: Average over IoU thresholds [0.5, 0.55, ..., 0.95]

In [ ]:
# ============================================================
# AVERAGE PRECISION (AP) - NumPy Implementation
# ============================================================

def calculate_ap(precisions, recalls):
    """
    Calculate Average Precision from precision-recall curve.
    
    Uses 11-point interpolation (PASCAL VOC style).
    """
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    
    # Sort by recall
    sorted_idx = np.argsort(recalls)
    recalls = recalls[sorted_idx]
    precisions = precisions[sorted_idx]
    
    # 11-point interpolation
    ap = 0
    for t in np.linspace(0, 1, 11):
        # Get max precision at recall >= t
        mask = recalls >= t
        if np.any(mask):
            ap += np.max(precisions[mask])
    
    return ap / 11

def calculate_map(all_predictions, all_ground_truths, iou_threshold=0.5):
    """
    Calculate mAP across all classes.
    
    Args:
        all_predictions: Dict {class_id: [(box, score), ...]}
        all_ground_truths: Dict {class_id: [box, ...]}
    """
    aps = []
    
    for class_id in all_ground_truths.keys():
        preds = all_predictions.get(class_id, [])
        gts = all_ground_truths[class_id]
        
        if len(preds) == 0 or len(gts) == 0:
            aps.append(0)
            continue
        
        # Sort predictions by score
        preds = sorted(preds, key=lambda x: x[1], reverse=True)
        
        # Calculate precision/recall at each threshold
        precisions = []
        recalls = []
        
        for i in range(1, len(preds) + 1):
            pred_boxes = [p[0] for p in preds[:i]]
            p, r, _ = calculate_precision_recall(pred_boxes, gts, iou_threshold)
            precisions.append(p)
            recalls.append(r)
        
        ap = calculate_ap(precisions, recalls)
        aps.append(ap)
    
    return np.mean(aps)

print("✅ mAP calculation functions defined")

In [ ]:
# Visualize Precision-Recall Curve
def plot_precision_recall_curve():
    """Sample precision-recall curve."""
    # Simulated values
    recalls = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
    precisions = np.array([1.0, 0.95, 0.9, 0.85, 0.8, 0.7, 0.6, 0.5, 0.3, 0.1])
    
    ap = calculate_ap(precisions, recalls)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.plot(recalls, precisions, 'b-', linewidth=2, marker='o', label=f'AP = {ap:.3f}')
    ax.fill_between(recalls, precisions, alpha=0.3)
    
    ax.set_xlabel('Recall', fontsize=12)
    ax.set_ylabel('Precision', fontsize=12)
    ax.set_title('📈 Precision-Recall Curve', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'pr_curve.png', dpi=150)
    plt.show()

plot_precision_recall_curve()

## 📝 Summary

### Implemented Metrics (NumPy only):

| Metric | Formula | Function |
|--------|---------|----------|
| **IoU** | Intersection/Union | `calculate_iou()` |
| **NMS** | Suppress overlapping boxes | `nms()` |
| **Precision** | TP/(TP+FP) | `calculate_precision_recall()` |
| **Recall** | TP/(TP+FN) | `calculate_precision_recall()` |
| **AP** | Area under PR curve | `calculate_ap()` |
| **mAP** | Mean AP across classes | `calculate_map()` |

### Next: Task 7 - YOLO Architecture Deep Dive

In [ ]:
print("\n" + "="*60)
print("✅ TASK 6 COMPLETE: Object Detection Metrics")
print("="*60)
print("\n📋 Implemented (NumPy only):")
print("   ✓ IoU calculation")
print("   ✓ Non-Maximum Suppression (NMS)")
print("   ✓ Precision, Recall, F1-Score")
print("   ✓ Average Precision (AP)")
print("   ✓ Mean Average Precision (mAP)")
print("\n➡️ Ready for Task 7: YOLO Architecture")